<div style="text-align:center"><a href="https://colab.research.google.com/github/PexMor/jupyter-playground/blob/main/BetterTabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br/>
<a style="display:inline-block" href="https://github.com/PexMor/jupyter-playground/blob/main/BetterTabs.ipynb" target="_parent"><img alt="GitHub forks" src="https://img.shields.io/github/forks/PexMor/jupyter-playground?label=fork%20me&logo=github&style=plastic"></a></div>

# Better tables in Jupyter notebooks

Purpose of this notebook is to show an option to get beyond the `print` inside Jyputer when producing the tabular content. As the Jupyter is embedded inside HTML and it offers the capability to use HTML for formating among other options then making more visually appealing tables is great option for more legible data formating.

## The code

Let's start with some code hygiene, capture versions and environment so anyone reading this has better starting position.

In [1]:
!pip -q install watermark tabulate

In [2]:
import watermark

%load_ext watermark

Record the versions into the notebook output

In [3]:
%watermark -i -n -v -m -iv

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.25.0

Compiler    : Clang 12.0.5 (clang-1205.0.22.9)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

watermark: 2.2.0



The core of this notebook is the following class `TabOut` which does the table construction over time (i.e. in loop).

In [4]:
from IPython import display
import tabulate
import datetime
class TabOut:
    def __init__(self, headers, show_rel_ts = True, show_abs_ts = True):
        self.headers = headers
        self.show_rel_ts = show_rel_ts
        self.show_abs_ts = show_abs_ts
        if self.show_rel_ts:
            self.headers.append("Rel.time[s]")
        if self.show_abs_ts:
            self.headers.append("Abs.time[s]")
        self.data = []
        self.last_ts = self.base_ts = datetime.datetime.now()
        
    def display(self, row, text = "In Progress!"):
        loc_row = row.copy()
        now_ts = datetime.datetime.now()
        dur_rel = (now_ts - self.last_ts).total_seconds()
        dur_abs = (now_ts - self.base_ts).total_seconds()
        self.last_ts = now_ts
        if self.show_rel_ts:
            loc_row.append(dur_rel)
        if self.show_abs_ts:
            loc_row.append(dur_abs)
        self.data.append(loc_row)
        self.show(text)
    
    def clear(self):
        self.data = []
        
    def show(self, text):
        display.clear_output(wait=True)
        html = f"<h2>{text}</h2>"
        html += tabulate.tabulate(self.data, tablefmt='html',headers=self.headers)
        html += f"<h2>{text}</h2>"
        display.display(display.HTML(html))

    def done(self, text = "Done!"):
        self.show(text)

## Demo time

In [5]:
import time
table = [["Sun",696000,1989100000],
         ["Earth",6371,5973.6],
         ["Moon",1737,73.5],
         ["Mars",3390,641.85]]
out = TabOut(headers=["Planet","R (km)", "mass (x 10^29 kg)"])
for no in range(len(table)*10):
    out.display(table[no%len(table)])
    time.sleep(0.3)
out.done()

Planet,R (km),mass (x 10^29 kg),Rel.time[s],Abs.time[s]
Sun,696000,1.9891e+09,6.8e-05,6.8e-05
Earth,6371,5973.6,0.313822,0.31389
Moon,1737,73.5,0.306011,0.619901
Mars,3390,641.85,0.302768,0.922669
Sun,696000,1.9891e+09,0.308229,1.2309
Earth,6371,5973.6,0.308771,1.53967
Moon,1737,73.5,0.30959,1.84926
Mars,3390,641.85,0.30884,2.1581
Sun,696000,1.9891e+09,0.308263,2.46636
Earth,6371,5973.6,0.309143,2.7755
